In [1]:
import django
django.setup()
from nagg.models import NewsItemCollection, NewsItem, NewsItemCollectionMembership

In [2]:
from brains import spark_common
from pyspark import SparkContext, SparkConf, SQLContext

In [3]:
import psycopg2
import sys

In [4]:
sql = """
SELECT "nagg_newsitemcollectionmembership"."id", "nagg_newsitemcollectionmembership"."newsitem_id",
       "nagg_newsitemcollectionmembership"."newsitemcollection_id", "nagg_newsitemcollectionmembership"."data"::text,
       "nagg_newsitem"."source", "nagg_newsitem"."url", "nagg_newsitem"."text",
       "nagg_newsitem"."publish_date", "nagg_newsitem"."retrieval_date", "nagg_newsitem"."search_index"::text,
       "nagg_newsitemcollection"."name", "nagg_newsitemcollection"."metadata"::text
FROM "nagg_newsitemcollectionmembership"
  INNER JOIN "nagg_newsitemcollection"
  ON ("nagg_newsitemcollectionmembership"."newsitemcollection_id" = "nagg_newsitemcollection"."id")
  INNER JOIN "nagg_newsitem"
  ON ("nagg_newsitemcollectionmembership"."newsitem_id" = "nagg_newsitem"."id")
    WHERE "nagg_newsitemcollectionmembership"."newsitemcollection_id" = 2
    """

In [ ]:
with spark_common.SparkAPP('ipynb spartapp exp') as app:
    #nic = NewsItemCollection.objects.get(id=2)
    #qs = NewsItemCollection.objects.filter(newsitemcollectionmembership__newsitemcollection_id=2)
    #qs = qs.newsitemcollectionmembership_set.select_related()
    #qs.query.add_fields(['text', 'source'], True)
    #qs.extra(select={'membership_id': '"nagg_newsitemcollectionmembership"."id"'})
    #df = app.data_from_from_sql(qs[:1000])

    df = app.data_from_from_sql(sql)
    df.show()
    print(df.count())
    rdd = df.map(lambda x: (x.id, guess_lang(x.text)[0][0]))
    print(rdd.collect())

In [5]:
#del app
app = spark_common.SparkAPP('save exp')
df = app.data_from_from_sql(sql)

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.4/logging/__init__.py", line 982, in emit
    self.flush()
  File "/usr/lib/python3.4/logging/__init__.py", line 962, in flush
    self.stream.flush()
RuntimeError: reentrant call inside <_io.BufferedWriter name='/dream/codes/nagg/debug.log'>
Call stack:
  File "/usr/lib/python3.4/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.4/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/daniel/.virtualenvs/naggenv/lib/python3.4/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/daniel/.virtualenvs/naggenv/lib/python3.4/site-packages/traitlets/config/application.py", line 592, in launch_instance
    app.start()
  File "/home/daniel/.virtualenvs/naggenv/lib/python3.4/site-packages/ipykernel/kernelapp.py", line 403, in start
    ioloop.IOLoop.instance().start()
  File "/home/daniel/.virtualenvs/

In [13]:
rdd = df.map(lambda x: (x.id, guess_lang(x.text)[0][0]))
rdd = rdd.repartition(3)

In [23]:
def worker_update_results(idx, iterator):
    """Updates data on the DB, each worker in their own
    transaction.
    
    If we require consitency over the full result set, we 
    need to employ a two phase commit.
    """
    conn = psycopg2.connect(database="naggdb", user="nagg", password="nagg", host="10.0.3.7")
    try:
        c = conn.cursor()
        c.executemany("""UPDATE nagg_newsitemcollectionmembership SET data = %s WHERE id = %s;""",
              (('{"value": "%s"}' % (_[1],), _[0]) for _ in iterator))
    except Exception as ex:
        conn.rollback()
        yield False, 'partition {} rollback: {}'.format(idx, ex)
    else:
        conn.commit()
        yield True, 'partition {} OK'.format(idx)
#    yield str(conn)
#    yield idx, [('{"value": "%s"}' % (_[1],), _[0]) for _ in iterator]

res = rdd.mapPartitionsWithIndex(worker_update_results)
res.collect()

[(True, 'partition 0 OK'), (True, 'partition 1 OK'), (True, 'partition 2 OK')]

In [ ]:
# this does not work with sqlcontext, only hivecontext
qs = nic.items.filter(newsitemcollection__id=2)[:1000]
qs.query.add_fields(['text', 'source', 'id'], True)
df = app.data_from_from_sql(qs)
writer = df.write.format('jdbc').options(
            url='jdbc:postgresql://10.0.3.7/naggdb?user=nagg&password=nagg')
writer.saveAsTable(app.sc.applicationId.replace('-', '_'))

In [ ]:
app.sc.applicationId

In [9]:
import nltk

In [10]:
stopwords = {}
for lang in ['danish', 'dutch', 'english', 'finnish', 'french', 'german', 'hungarian',
             'italian', 'norwegian', 'portuguese', 'russian', 'spanish', 'swedish',
             'turkish']:
    stopwords[lang] = set(nltk.corpus.stopwords.words(lang))

for lang, words in stopwords.items():
    print(lang, len(words))

swedish 114
hungarian 198
portuguese 203
finnish 229
german 231
norwegian 172
turkish 53
dutch 101
spanish 313
danish 94
italian 279
french 155
english 127
russian 151


In [11]:
def guess_lang(text, n=2):
    doc_words = set(nltk.word_tokenize(text))
    scores = {}
    for lang, sw_words in stopwords.items():
        scores[lang] = len(sw_words.intersection(doc_words))
    return [(l, s) for l, s in sorted(scores.items(), key=lambda x: -x[1])][0:n]

In [ ]:
print(guess_lang('Hallo mijn naam is James'), 'dutch')
print(guess_lang('Hello my name is James'), 'english')
print(guess_lang('Hallo mein Name ist James'), 'german')
print(guess_lang('Hola mi nombre es James'), 'spanish')
print(guess_lang('Hej mitt namn är James'), 'swedish')
print(guess_lang('Hei mitt navn er James'), 'norwegian')
print(guess_lang('Hei, nimeni on James'), 'finish')
print(guess_lang("Bonjour je m'appelle James"), 'french')
print(guess_lang('Hej mit navn er James'), 'danish')
print(guess_lang("Привет, меня зовут Джеймс"), 'russian')
print(guess_lang("Ciao il mio nome è James"), 'italian')
print(guess_lang("Hello a nevem James"), 'hungarian')
print(guess_lang("Olá meu nome é James"), 'portuguese')

In [ ]:
texts = {
    'french': """Un homme abattu et tué sa femme et deux autres à son domicile de Los Angeles le réveillon du Nouvel An avant que son fils a lutté le pistolet loin et fatalement lui a tiré dans une chaîne d'événements apparemment déclenchées par un différend sur une machine à laver.

Les deux autres victimes tuées étaient de 48 ans, ancienne petite amie de son fils qui vit aussi à la maison dans la région Rowland Heights de la ville et un homme de 27 ans qui était en visite, le département de thecounty shérif.""",
    'english': """A man shot and killed his wife and two others in his home in Los Angeles on New Year’s Eve before his son wrestled the gun away and fatally shot him in a chain of events apparently set off by a dispute over a washing machine. 

The two other victims killed were the son’s 48-year-old girlfriend who also lives at the house in the Rowland Heights area of the city and a 27-year-old man who was visiting, thecounty sheriff’s department said.""",
    'dutch': """Een man schoot en doodde zijn vrouw en twee anderen in zijn huis in Los Angeles op oudejaarsavond voor zijn zoon worstelde het pistool weg en doodgeschoten hem in een keten van gebeurtenissen blijkbaar gecompenseerd door een geschil over een wasmachine.

De twee andere slachtoffers waren gedood, de zoon van de 48-jarige vriendin die ook woont in het huis in het Rowland Heights gebied van de stad en een 27-jarige man die op bezoek was, afdeling thecounty sheriff gezegd.
""",
    'german': """Ein Mann erschoss seine Frau und zwei andere in seinem Haus in Los Angeles am Silvesterabend, bevor sein Sohn rangen die Waffe weg und erschossen ihn in eine Kette von Ereignissen offenbar durch einen Streit über eine Waschmaschine setzen.

Die beiden anderen Opfer getötet wurden, waren des Sohnes 48-jährige Freundin, die auch lebt im Haus in der Rowland Heights Bereich der Stadt und einem 27-jährigen Mann, der zu Besuch war, sagte Abteilungs thecounty Sheriffs.""",
    'spanish': """Un hombre disparó y mató a su esposa y otros dos en su casa de Los Angeles en la víspera de Año Nuevo antes de que su hijo luchó el arma y le disparó fatalmente en una cadena de acontecimientos aparentemente partió por una disputa sobre una lavadora.

Las otras dos víctimas asesinadas eran 48 años de edad, la novia del hijo, que también vive en la casa en la zona de Rowland Heights de la ciudad y un hombre de 27 años de edad, que estaba de visita, dijo el departamento del sheriff thecounty.""",
    'swedish': """En man sköt och dödade hans fru och två andra i sitt hem i Los Angeles på nyårsafton innan hans son brottades pistolen bort och dödligt sköt honom i en kedja av händelser tydligen iväg av en tvist om en tvättmaskin.

De två andra offer dödade var sonens 48-åriga flickvän som också bor i huset i området Rowland Heights i staden och en 27-årig man som besökte, sade thecounty sheriff avdelning.""",
    'norwegian': """En mann skjøt og drepte sin kone og to andre i sitt hjem i Los Angeles på nyttårsaften før sønnen kjempet pistolen bort og skjebnesvangert skjøt ham i en kjede av hendelser tilsynelatende utløst av en krangel om en vaskemaskin.

De to andre ofrene som ble drept var sønnens 48 år gamle kjæreste som også bor i huset i Rowland Heights-området av byen og en 27-år gammel mann som var på besøk, sa thecounty sheriff avdeling.""",
    'danish': """En mand skød og dræbte sin kone og to andre i sit hjem i Los Angeles den nytårsaften, før hans søn kæmpede pistolen væk og dødeligt skudt ham i en kæde af begivenheder tilsyneladende sat ud af en tvist over en vaskemaskine.

De to andre ofre dræbte var sønnens 48-årige kæreste, der også bor i huset i Rowland Heights område af byen og en 27-årig mand, der var på besøg, sagde thecounty sheriffens afdeling.""",
}

In [ ]:
for lang in texts:
    guess = guess_lang(texts[lang])
    print(int(guess[0][0]==lang) , lang, guess)

In [ ]:
nic.items.filter(newsitemcollection__id=1).count()

In [25]:
conn

NameError: name 'conn' is not defined